<a href="https://colab.research.google.com/github/nischalraii/summarization-t5-sm/blob/main/summarization_t5_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
str = """Nepal's last major incident involving a commercial flight was in January 2023, when a Yeti Airlines service crashed while landing at Pokhara, killing all 72 aboard . the incident comes two weeks after a plane crash in Kathmandu killed all 18 aboard, including the pilot and four Chinese tourists ."""

In [2]:
str

"Nepal's last major incident involving a commercial flight was in January 2023, when a Yeti Airlines service crashed while landing at Pokhara, killing all 72 aboard . the incident comes two weeks after a plane crash in Kathmandu killed all 18 aboard, including the pilot and four Chinese tourists ."

In [3]:
!pip install transformers datasets evaluate rouge_score
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7456561e28f7e806f1993da78ef57e48fd7baff1169bf070d7fd625403fddc1c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pya

In [4]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [5]:
billsum = billsum.train_test_split(test_size=0.2)

In [6]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [7]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 9084 of the Elections Code is amended to read:\n9084.\nThe ballot pamphlet shall contain all of the following:\n(a) A complete copy of each state measure.\n(b) A copy of the specific constitutional or statutory provision, if any, that each state measure would repeal or revise.\n(c) A copy of the arguments and rebuttals for and against each state measure.\n(d) A copy of the analysis of each state measure.\n(e) Tables of contents, indexes, art work, graphics, and other materials that the Secretary of State determines will make the ballot pamphlet easier to understand or more useful for the average voter.\n(f) A notice, conspicuously printed on the cover of the ballot pamphlet, indicating that additional copies of the ballot pamphlet will be mailed by the county elections official upon request.\n(g) A written explanation of the judicial retention procedure as required by Section 9083.\n(h) The Vo

In [8]:


checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [9]:


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [10]:
import evaluate

rouge = evaluate.load("rouge")


In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="./summarization-t5-sm",  # Save model locally
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.878562,0.118800,0.031200,0.099800,0.099900,19.000000
2,No log,2.667876,0.128500,0.040200,0.105700,0.105600,19.000000
3,No log,2.606250,0.132000,0.042600,0.109300,0.109300,19.000000
4,No log,2.589978,0.133800,0.045700,0.111000,0.110900,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=248, training_loss=3.0022979244109123, metrics={'train_runtime': 296.092, 'train_samples_per_second': 13.361, 'train_steps_per_second': 0.838, 'total_flos': 1070824333246464.0, 'train_loss': 3.0022979244109123, 'epoch': 4.0})

In [18]:
model.save_pretrained("./summarization-t5-sm")
tokenizer.save_pretrained("./summarization-t5-sm")

('./summarization-t5-sm/tokenizer_config.json',
 './summarization-t5-sm/special_tokens_map.json',
 './summarization-t5-sm/spiece.model',
 './summarization-t5-sm/added_tokens.json',
 './summarization-t5-sm/tokenizer.json')

In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("./summarization-t5-sm")
tokenizer = AutoTokenizer.from_pretrained("./summarization-t5-sm")

In [20]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [21]:
text = """4 Chinese Passengers, Pilot Killed In Helicopter Crash In Nepal
Nepal's last major incident involving a commercial flight was in January 2023, when a Yeti Airlines service crashed while landing at Pokhara, killing all 72 aboard.
World NewsAgence France-PresseUpdated: August 07, 2024 8:01 pm IST
4 Chinese Passengers, Pilot Killed In Helicopter Crash In Nepal
This incident comes two weeks after a plane crash in Kathmandu killed 18 aboard. (Representational)
Kathmandu, Nepal: A helicopter crashed in central Nepal on Wednesday, killing all five people aboard including the pilot and four Chinese tourists.
Nepal has a woeful track record on aviation safety and the Himalayan republic has seen a spate of deadly light plane and helicopter crashes over the decades.
The Air Dynasty chopper was heading to Syabrubesi, a starting point for many trekking routes popular with tourists, from the capital Kathmandu.
It lost contact about three minutes after taking off, a press release from the Civil Aviation Authority said, and crashed in Nuwakot district north of the capital.
Another helicopter was dispatched immediately to the accident site for rescue, it said.
"All five people aboard including the pilot are dead," police spokesman Dan Bahadur Karki told AFP.
The helicopter lost contact about three minutes after taking off, authorities said
The helicopter lost contact about three minutes after taking off, authorities said
Nuwakot district officer Ram Krishna Adhikari said five dead bodies have been retrieved from the accident site.
"It has crashed in a jungle area on the slope of a hill. We do not know the cause yet or how it happened," he said from the crash site.
Adhikari said that police and army personnel were dispatched after locals had alerted him about the crash and a fire at the site.
Nepal's air industry has boomed in recent years, carrying goods and people between hard-to-reach areas as well as foreign trekkers and climbers.
But it has been plagued by poor safety due to insufficient training and maintenance, issues compounded by mountainous Nepal's treacherous geography.
The Himalayan republic has some of the world's most remote and tricky runways, flanked by snow-capped peaks with approaches that pose a challenge even for accomplished pilots.
The European Union has banned all Nepali carriers from its airspace over safety concerns.
Wednesday's incident comes two weeks after a plane crash in Kathmandu killed all 18 aboard except the pilot.
Six people, including five members of a Mexican family, were killed when a helicopter crashed soon after take-off in the Everest region in 2023.
Nepal's tourism minister Rabindra Adhikari was among seven killed in a helicopter crash in 2019.
The country's last major incident involving a commercial flight was in January 2023 when a Yeti Airlines service crashed while landing at Pokhara, killing all 72 aboard.
That was Nepal's deadliest since 1992 when all 167 people aboard a Pakistan International Airlines plane were killed when it crashed on approach to Kathmandu airport.
A Thai Airways aircraft crashed near the same airport earlier that year, killing 113 people.

"""
summary = summarizer(text)
print(summary[0]['summary_text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (710 > 512). Running this sequence through the model will result in indexing errors


Nepal's last major incident involving a commercial flight was in January 2023, when a Yeti Airlines service crashed while landing at Pokhara, killing all 72 aboard . the incident comes two weeks after a plane crash in Kathmandu killed all 18 aboard, including the pilot and four Chinese tourists .


In [23]:
from huggingface_hub import login

login()


In [24]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# Load your model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./summarization-t5-sm')
tokenizer = AutoTokenizer.from_pretrained('./summarization-t5-sm')

# Define a model repository name and path
repo_name = "Ferreus/summarization-tm-small"

# Push the model to the Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at ./summarization-t5-sm and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/243M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ferreus/summarization-tm-small/commit/6c74278f7991345c7d5b3b03c45048a2b4f3ba25', commit_message='Upload tokenizer', commit_description='', oid='6c74278f7991345c7d5b3b03c45048a2b4f3ba25', pr_url=None, pr_revision=None, pr_num=None)